In [135]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
import joblib                                                                                                                                            
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import auc
from sklearn.preprocessing import MinMaxScaler

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from scipy.stats import pearsonr



In [136]:

fajiao = pd.read_table("../data-2/fajiao-condition-4.11.txt",sep='\t',index_col=0)
fajiao = pd.get_dummies(fajiao, columns=fajiao.columns) 
fajiao['label'] = fajiao.groupby(fajiao.columns.tolist()).ngroup()

fajiao_drop = fajiao.drop_duplicates()
fajiao_drop = fajiao_drop.drop('label', axis=1)

X = np.array(fajiao_drop)

# X  = StandardScaler().fit_transform(X)
X  = Normalizer().fit_transform(X)
# # X  = MinMaxScaler().fit_transform(X)
# X  = StandardScaler().fit_transform(X)
X.shape

(13, 17)

In [137]:
data = pd.read_table('../data-2/total4.15.txt',sep='\t',index_col=0)
data_fjg= data.loc[:, ~data.columns.str.contains('BC|NT|MC|PC')].T
data_fjg.columns = data_fjg.iloc[0]
data_fjg = data_fjg[1:]
data_fjg = data_fjg.sort_index()
data_fjg = data_fjg.replace("#DIV/0!", 0)
data_fjg = data_fjg.apply(pd.to_numeric)
data_fjg.fillna(0, inplace=True) 
# data_fjg


data_fjg.index = data_fjg.index.astype(float) 
data_fjg = data_fjg.sort_index() 
selected_columns = ['17431-03-7', '505-10-2', '74685-45-3', '622-78-6', '60-12-8', '104-46-1', '98-85-1']
data_fjg = data_fjg.loc[:, selected_columns]
# data_fjg

data_fjg['label'] = fajiao['label']
fajiao_mean = data_fjg.groupby('label').mean()

fajiao_mean_unique = fajiao_mean.reset_index().drop_duplicates()
fajiao_mean_unique = fajiao_mean_unique.drop('label', axis=1)
fajiao_mean_unique

Y = np.array(fajiao_mean_unique)
# 
Y  = Normalizer().fit_transform(Y)
# Y  = MinMaxScaler().fit_transform(Y)
# Y  = StandardScaler().fit_transform(Y)
# Y

In [139]:
from sklearn.neighbors import KNeighborsRegressor
base_model = KNeighborsRegressor(n_neighbors=5)

In [140]:
gbr_model = MultiOutputRegressor(base_model)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

MSE = []
RMSE = []
MAE = []
PCC = []
R2 = []
adjusted_r2 = []

MSE_train = []
RMSE_train = []
MAE_train = []
PCC_train = []
R2_train = []
adjusted_r2_train = []


for train_idx, test_idx in cv.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, y_test = Y[train_idx], Y[test_idx]
    gbr_model.fit(X_train, Y_train)
    # print(X_train.shape, Y_train.shape)
    #train
    x_pred = gbr_model.predict(X_train)
    mse = mean_squared_error(Y_train, x_pred)
    rmse = np.sqrt(mean_squared_error(Y_train, x_pred))
    mae = np.mean(np.abs(Y_train - x_pred))
    correlation_array = np.empty((len(Y_train), 1))
    for i in range(len(Y_train)):
        correlation = np.corrcoef(Y_train[i], x_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(Y_train, x_pred) 
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(Y_train) - 1) / (len(Y_train) - num_features - 1)
    MSE_train .append(mse)
    RMSE_train .append(rmse)
    MAE_train .append(mae)
    PCC_train .append(pcc)
    R2_train .append(r2)
    adjusted_r2_train .append(adj_r2)

    #test
    y_pred = gbr_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = np.mean(np.abs(y_test - y_pred))
    correlation_array = np.empty((len(y_test), 1))
    for i in range(len(y_test)):
        correlation = np.corrcoef(y_test[i], y_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(y_test, y_pred) 
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)
    MSE.append(mse)
    RMSE.append(rmse)
    MAE.append(mae)
    PCC.append(pcc)
    R2.append(r2)
    adjusted_r2.append(adj_r2)


for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE_train, RMSE_train,MAE_train, PCC_train,R2_train,adjusted_r2_train), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE_train) / len(MSE_train)
average_RMSE = sum(RMSE_train) / len(RMSE_train)
average_MAE= sum(MAE_train) / len(MAE_train)
average_PCC = sum(PCC_train) / len(PCC_train)
average_R2 = sum(R2_train) / len(R2_train)
average_adjusted_r2 = sum(adjusted_r2_train) / len(adjusted_r2_train)

print(f'Average MSE_train = {average_MSE:.7f}')
print(f'Average RMSE_train = {average_RMSE:.7f}')
print(f'Average MAE_train = {average_MAE:.7f}')
print(f'Average PCC_train = {average_PCC:.7f}')
# print(f'Average R2_train = {average_R2:.7f}')
# print(f'Average adjusted_R2_train = {average_adjusted_r2:.7f}')



for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE, RMSE,MAE, PCC,R2,adjusted_r2), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE) / len(MSE)
average_RMSE = sum(RMSE) / len(RMSE)
average_MAE= sum(MAE) / len(MAE)
average_PCC = sum(PCC) / len(PCC)
average_R2 = sum(R2) / len(R2)
average_adjusted_r2 = sum(adjusted_r2) / len(adjusted_r2)

print(f'Average MSE = {average_MSE:.7f}')
print(f'Average RMSE = {average_RMSE:.7f}')
print(f'Average MAE = {average_MAE:.7f}')
print(f'Average PCC = {average_PCC:.7f}')
# print(f'Average R2 = {average_R2:.7f}')
# print(f'Average adjusted_R2 = {average_adjusted_r2:.7f}')

Fold 1: MSE = 0.0047, RMSE = 0.0686,MAE = 0.0331, PCC = 0.9799, R2 = 0.2299, adjusted_r2 = -1.3103
Fold 2: MSE = 0.0087, RMSE = 0.0935,MAE = 0.0505, PCC = 0.9619, R2 = 0.1895, adjusted_r2 = -1.4314
Fold 3: MSE = 0.0062, RMSE = 0.0788,MAE = 0.0409, PCC = 0.9727, R2 = 0.2653, adjusted_r2 = -1.2041
Fold 4: MSE = 0.0080, RMSE = 0.0894,MAE = 0.0486, PCC = 0.9658, R2 = 0.1677, adjusted_r2 = -1.0808
Fold 5: MSE = 0.0081, RMSE = 0.0901,MAE = 0.0494, PCC = 0.9649, R2 = 0.2738, adjusted_r2 = -0.8156
Average MSE_train = 0.0071536
Average RMSE_train = 0.0840788
Average MAE_train = 0.0445109
Average PCC_train = 0.9690515
Fold 1: MSE = 0.0209, RMSE = 0.1444,MAE = 0.0757, PCC = 0.9066, R2 = -1.9249, adjusted_r2 = 2.4625
Fold 2: MSE = 0.0040, RMSE = 0.0636,MAE = 0.0361, PCC = 0.9852, R2 = -2.6101, adjusted_r2 = 2.8050
Fold 3: MSE = 0.0129, RMSE = 0.1135,MAE = 0.0646, PCC = 0.9440, R2 = -1.4320, adjusted_r2 = 2.2160
Fold 4: MSE = 0.0072, RMSE = 0.0849,MAE = 0.0481, PCC = 0.9723, R2 = -3448.8723, adjust

In [142]:

yp = gbr_model.predict(X)
# np.savetxt('my_array.txt', yp, delimiter='\t')
yp.shape  #

yp  = Normalizer().fit_transform(yp)
yp = MinMaxScaler().fit_transform(yp)
print(yp.shape)
loaded_model = joblib.load("../code-2/result/SVM_Classifier-7f.joblib")
y_pred = loaded_model.predict(yp).tolist() 
y_pred 

(13, 7)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [141]:

df13 = fajiao.drop_duplicates()

df54 = pd.read_csv("fjg_label.csv",index_col=0)
df54['code'] = fajiao['label']
df54['mean'] = df54.groupby('code')['label'].transform('mean')
df54['mean_label'] = np.where(df54['mean'] > 0.6, 1, 0)
df54

df54 = df54[df54.index.isin(df13.index)]
truth_simi = df54['mean_label'].tolist()
truth_simi

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [145]:
print('precision（test）:',metrics.precision_score(truth_simi,y_pred))
print('recall（test）:',metrics.recall_score(truth_simi,y_pred))
print('Accuracy（test）:',metrics.accuracy_score(truth_simi,y_pred))
# print('F1 （test）:',f1_score(truth_simi,y_pred))
print("y_test:",truth_simi,"\ny_pred:",y_pred)
fpr, tpr, thresholds = roc_curve(truth_simi, y_pred)
auc_1 = auc(fpr, tpr)
print("AUC : ", auc_1)

precision（测试集）: 0.0
recall（测试集）: 0.0
F1值（测试集）: 0.0
Accuracy: 0.9230769230769231
y_test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 
y_pred: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
AUC :  0.5


/data/public/dulin/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [146]:
yp
result = pd.DataFrame(yp,index = fajiao_drop.index,columns=selected_columns)
result

,17431-03-7,505-10-2,74685-45-3,622-78-6,60-12-8,104-46-1,98-85-1
4.2,0.529725,1.000000,2.220446e-16,0.486483,0.823003,0.0,0.817478
5.2,0.529725,1.000000,0.000000e+00,0.486483,0.823003,0.0,0.817478
6.2,0.000000,0.576929,3.237633e-01,0.044627,0.982615,0.0,0.378104
16.1,1.000000,0.928515,3.284288e-01,0.938885,0.423196,0.0,1.000000
17.1,0.420340,0.565718,8.623452e-01,1.000000,0.000000,0.0,0.996488
23.1,0.742554,0.057710,1.000000e+00,0.119035,0.520065,0.0,0.179685
24.1,0.066197,0.000000,4.643763e-01,0.000000,1.000000,0.0,0.072180
25.1,0.516817,0.507296,3.426897e-01,0.734459,0.889180,0.0,0.509709
26.1,0.332150,0.255072,4.507104e-01,0.469111,0.817237,0.0,0.507803
27.1,0.276195,0.360262,7.678516e-01,0.701267,0.773214,0.0,0.000000
